In [1]:
# from usd_writer import write_usd
# import json

# with open("o2tank.json", "r") as f:
#     data = json.load(f)

# a = write_usd(model=data["model"], output_path="o2tank.usd")

Script that writes the .usda from o2tank.json 

In [2]:
# from __future__ import annotations

# import ast
# import re
# from typing import Any, Dict, List, Optional, Tuple

# from pxr import Usd, UsdGeom, Sdf, Gf


# # -------------------------
# # Helpers: parsing "rhs" strings from your partner's JSON
# # -------------------------

# def _strip_comments_and_semicolons(s: str) -> str:
#     # Partner's strings sometimes contain comments/semicolons in other exports;
#     # this keeps parsing resilient.
#     s = re.sub(r"//.*?$", "", s, flags=re.MULTILINE)  # remove // comments
#     s = s.replace(";", " ")
#     return s.strip()

# def parse_int_list(rhs: str) -> List[int]:
#     """
#     Parses strings like: "[4,4,4,4,4,4]" into [4,4,4,4,4,4].
#     """
#     s = _strip_comments_and_semicolons(rhs)
#     # ast.literal_eval is safe for Python literals
#     val = ast.literal_eval(s)
#     if not isinstance(val, (list, tuple)):
#         raise ValueError(f"Expected list/tuple for int list, got: {type(val)} from {rhs!r}")
#     return [int(x) for x in val]

# def parse_points_list(rhs: str) -> List[Gf.Vec3f]:
#     """
#     Parses strings like:
#       "[(-1, -1, -1),( 1, -1, -1), ... ]"
#     into a list of Gf.Vec3f.
#     """
#     s = _strip_comments_and_semicolons(rhs)
#     val = ast.literal_eval(s)  # yields list/tuple of tuples
#     if not isinstance(val, (list, tuple)):
#         raise ValueError(f"Expected list/tuple for points, got: {type(val)} from {rhs!r}")
#     pts: List[Gf.Vec3f] = []
#     for p in val:
#         if not (isinstance(p, (list, tuple)) and len(p) == 3):
#             raise ValueError(f"Bad point element {p!r} in {rhs!r}")
#         pts.append(Gf.Vec3f(float(p[0]), float(p[1]), float(p[2])))
#     return pts


# # -------------------------
# # Helpers: walking your partner's JSON AST-ish format
# # -------------------------

# def find_construct(root: Any, *, keyword: str, name: Optional[str] = None) -> Optional[Dict[str, Any]]:
#     """
#     Recursively find the first construct dict matching keyword (and optionally name).
#     Partner format uses dicts like:
#       {"kind":"construct","keyword":"part","name":"O2Tank","body":[...]}
#     """
#     if isinstance(root, dict):
#         if root.get("kind") == "construct" and root.get("keyword") == keyword:
#             if name is None or root.get("name") == name:
#                 return root
#         # descend
#         for k, v in root.items():
#             found = find_construct(v, keyword=keyword, name=name)
#             if found:
#                 return found
#     elif isinstance(root, list):
#         for item in root:
#             found = find_construct(item, keyword=keyword, name=name)
#             if found:
#                 return found
#     return None

# def find_child_construct(parent: Dict[str, Any], *, keyword: str, name: Optional[str] = None) -> Optional[Dict[str, Any]]:
#     body = parent.get("body", [])
#     for node in body:
#         if isinstance(node, dict) and node.get("kind") == "construct" and node.get("keyword") == keyword:
#             if name is None or node.get("name") == name:
#                 return node
#     return None

# def collect_assignments(node: Dict[str, Any]) -> Dict[str, Any]:
#     """
#     Collect assignments inside a node body:
#       {"kind":"assignment","lhs":"points","rhs":"[...]"}
#     Returns dict {lhs: rhs}.
#     """
#     out: Dict[str, Any] = {}
#     for child in node.get("body", []):
#         if isinstance(child, dict) and child.get("kind") == "assignment":
#             out[child["lhs"]] = child["rhs"]
#     return out


# # -------------------------
# # USD Writing (sanity-check conversion)
# # -------------------------

# def write_usd_from_partner_json(
#     partner_json: List[Dict[str, Any]],
#     output_path: str = "o2tank_sanity.usda",
#     *,
#     root_prim_name: str = "O2Tank",
#     up_axis: str = "Z",
#     meters_per_unit: float = 1.0,
# ) -> str:
#     """
#     Creates a USD stage from your partner's current JSON export.
#     This is a *sanity check* converter:
#       - Fixes typed attributes: points/counts/indices become proper USD arrays
#       - Creates a Mesh prim under the O2Tank Xform
#       - Leaves composition intent (references/payloads) out as requested
#     """
#     stage = Usd.Stage.CreateNew(output_path)

#     # Stage-level metadata (Issue B)
#     stage.SetMetadata("metersPerUnit", float(meters_per_unit))
#     UsdGeom.SetStageUpAxis(stage, UsdGeom.Tokens.z if up_axis.upper() == "Z" else UsdGeom.Tokens.y)

#     # Find "part" named O2Tank
#     o2tank = find_construct(partner_json, keyword="part", name="O2Tank")
#     if not o2tank:
#         raise ValueError("Could not find construct: keyword='part', name='O2Tank'")

#     # Find nested "part" named "mesh" under O2Tank
#     mesh_part = find_child_construct(o2tank, keyword="part", name="mesh")
#     if not mesh_part:
#         raise ValueError("Could not find child construct: keyword='part', name='mesh' under O2Tank")

#     assigns = collect_assignments(mesh_part)

#     # Parse mesh data (Issue A)
#     if "points" not in assigns or "faceVertexCounts" not in assigns or "faceVertexIndices" not in assigns:
#         raise ValueError(f"Mesh assignments missing required fields. Found keys: {list(assigns.keys())}")

#     points = parse_points_list(assigns["points"])
#     counts = parse_int_list(assigns["faceVertexCounts"])
#     indices = parse_int_list(assigns["faceVertexIndices"])

#     # Create prims
#     root_path = Sdf.Path(f"/{root_prim_name}")
#     xform = UsdGeom.Xform.Define(stage, root_path)

#     mesh_path = root_path.AppendChild("mesh")
#     mesh = UsdGeom.Mesh.Define(stage, mesh_path)

#     # Author typed USD mesh attributes
#     mesh.CreatePointsAttr(points)
#     mesh.CreateFaceVertexCountsAttr(counts)
#     mesh.CreateFaceVertexIndicesAttr(indices)
#     mesh.CreateSubdivisionSchemeAttr(UsdGeom.Tokens.none)

#     # Optional: extents (helps viewport framing)
#     # Compute a simple AABB from points:
#     xs = [p[0] for p in points]
#     ys = [p[1] for p in points]
#     zs = [p[2] for p in points]
#     extent = [Gf.Vec3f(min(xs), min(ys), min(zs)), Gf.Vec3f(max(xs), max(ys), max(zs))]
#     mesh.CreateExtentAttr(extent)

#     # Preserve the original string payload as customData for trace/debug (optional)
#     # This keeps "by-hand fixes" minimal while not losing provenance.
#     mesh_prim = mesh.GetPrim()
#     mesh_prim.SetCustomDataByKey("src_points_rhs", assigns["points"])
#     mesh_prim.SetCustomDataByKey("src_faceVertexCounts_rhs", assigns["faceVertexCounts"])
#     mesh_prim.SetCustomDataByKey("src_faceVertexIndices_rhs", assigns["faceVertexIndices"])

#     # Nice-to-have: default prim for easier viewing
#     stage.SetDefaultPrim(xform.GetPrim())

#     stage.GetRootLayer().Save()
#     return output_path


# # -------------------------
# # Example usage
# # -------------------------
# if __name__ == "__main__":
#     # Replace this with reading your partner JSON from a file if needed.
#     partner_json = [
#       {
#         "kind": "construct",
#         "keyword": "package",
#         "name": "Oxygen Tank for Habitation",
#         "body": [
#           {
#             "kind": "construct",
#             "keyword": "part def",
#             "name": "Mesh",
#             "body": [
#               {"kind": "construct","keyword": "attribute","name": "points","type": "String"},
#               {"kind": "construct","keyword": "attribute","name": "faceVertexCounts","type": "String"},
#               {"kind": "construct","keyword": "attribute","name": "faceVertexIndices","type": "String"}
#             ]
#           },
#           {
#             "kind": "construct",
#             "keyword": "part",
#             "name": "O2Tank",
#             "body": [
#               {
#                 "kind": "construct",
#                 "keyword": "part",
#                 "name": "mesh",
#                 "type": "Mesh",
#                 "body": [
#                   {"kind": "assignment","lhs": "points","rhs": "[(-1, -1, -1),( 1, -1, -1),( 1,  1, -1),(-1,  1, -1),(-1, -1,  1),( 1, -1,  1),( 1,  1,  1),(-1,  1,  1)]"},
#                   {"kind": "assignment","lhs": "faceVertexCounts","rhs": "[4,4,4,4,4,4]"},
#                   {"kind": "assignment","lhs": "faceVertexIndices","rhs": "[0,1,2,3,4,5,6,7,0,1,5,4,3,2,6,7,0,3,7,4,1,2,6,5]"}
#                 ]
#               }
#             ]
#           }
#         ]
#       }
#     ]

#     out = write_usd_from_partner_json(partner_json, output_path="o2tank_sanity.usda")
#     print("Wrote:", out)


In [4]:
from __future__ import annotations

import os
from typing import Any, Dict, Tuple, List

from pxr import Usd, UsdGeom, UsdShade, Sdf, Gf, Vt


def ensure_parent_dir(path: str) -> None:
    parent = os.path.dirname(path)
    if parent:
        os.makedirs(parent, exist_ok=True)


def set_stage_metadata(stage: Usd.Stage, meters_per_unit: float = 1.0, up_axis: str = "Z") -> None:
    stage.SetMetadata("metersPerUnit", float(meters_per_unit))
    UsdGeom.SetStageUpAxis(stage, UsdGeom.Tokens.z if up_axis.upper() == "Z" else UsdGeom.Tokens.y)


def make_box_mesh_points_indices(dims_m: Tuple[float, float, float]):
    L, W, H = dims_m
    hx, hy, hz = L / 2.0, W / 2.0, H / 2.0

    pts = [
        Gf.Vec3f(-hx, -hy, -hz),
        Gf.Vec3f(+hx, -hy, -hz),
        Gf.Vec3f(+hx, +hy, -hz),
        Gf.Vec3f(-hx, +hy, -hz),
        Gf.Vec3f(-hx, -hy, +hz),
        Gf.Vec3f(+hx, -hy, +hz),
        Gf.Vec3f(+hx, +hy, +hz),
        Gf.Vec3f(-hx, +hy, +hz),
    ]

    counts = [4, 4, 4, 4, 4, 4]
    indices = [
        0, 1, 2, 3,
        4, 5, 6, 7,
        0, 1, 5, 4,
        3, 2, 6, 7,
        0, 3, 7, 4,
        1, 2, 6, 5,
    ]

    extent = [Gf.Vec3f(-hx, -hy, -hz), Gf.Vec3f(+hx, +hy, +hz)]
    return pts, counts, indices, extent


def write_geometry_layer(model: Dict[str, Any], geom_path: str, *, meters_per_unit: float = 1.0, up_axis: str = "Z") -> str:
    ensure_parent_dir(geom_path)
    stage = Usd.Stage.CreateNew(geom_path)
    set_stage_metadata(stage, meters_per_unit=meters_per_unit, up_axis=up_axis)

    dims = tuple(model["properties"]["dims_m"])
    points, counts, indices, extent = make_box_mesh_points_indices(dims)

    mesh = UsdGeom.Mesh.Define(stage, "/OxygenTank_Geom")
    mesh.CreatePointsAttr(points)
    mesh.CreateFaceVertexCountsAttr(counts)
    mesh.CreateFaceVertexIndicesAttr(indices)
    mesh.CreateSubdivisionSchemeAttr(UsdGeom.Tokens.none)
    mesh.CreateExtentAttr(extent)

    stage.SetDefaultPrim(mesh.GetPrim())
    stage.GetRootLayer().Save()
    return geom_path


def write_material_layer(model: Dict[str, Any], mtl_path: str, *, meters_per_unit: float = 1.0, up_axis: str = "Z") -> str:
    ensure_parent_dir(mtl_path)
    stage = Usd.Stage.CreateNew(mtl_path)
    set_stage_metadata(stage, meters_per_unit=meters_per_unit, up_axis=up_axis)

    material_name = model["properties"].get("material", "Material")
    mat_path = Sdf.Path(f"/Materials/{material_name}")

    material = UsdShade.Material.Define(stage, mat_path)
    shader = UsdShade.Shader.Define(stage, mat_path.AppendChild("PreviewSurface"))
    shader.CreateIdAttr("UsdPreviewSurface")

    shader.CreateInput("diffuseColor", Sdf.ValueTypeNames.Color3f).Set(Gf.Vec3f(0.7, 0.7, 0.7))
    shader.CreateInput("metallic", Sdf.ValueTypeNames.Float).Set(0.9)
    shader.CreateInput("roughness", Sdf.ValueTypeNames.Float).Set(0.35)

    material.CreateSurfaceOutput().ConnectToSource(shader.ConnectableAPI(), "surface")

    stage.SetDefaultPrim(material.GetPrim())
    stage.GetRootLayer().Save()
    return mtl_path


def set_customdata_requirements(prim: Usd.Prim, reqs: List[str]) -> None:
    """
    Strict USD builds require Vt.*Array types for customData arrays.
    """
    prim.SetCustomDataByKey("trace:requirements", Vt.StringArray([str(r) for r in reqs]))


def write_component_layer(model: Dict[str, Any], component_path: str, *, meters_per_unit: float = 1.0, up_axis: str = "Z") -> str:
    ensure_parent_dir(component_path)
    stage = Usd.Stage.CreateNew(component_path)
    set_stage_metadata(stage, meters_per_unit=meters_per_unit, up_axis=up_axis)

    name = model["name"]
    uid = model["id"]

    root = UsdGeom.Xform.Define(stage, f"/{name}")
    root_prim = root.GetPrim()

    # Typed scalar attributes
    root_prim.CreateAttribute("asset:id", Sdf.ValueTypeNames.String).Set(uid)
    root_prim.CreateAttribute("class:domain", Sdf.ValueTypeNames.String).Set(model["classification"]["domain"])
    root_prim.CreateAttribute("class:kind", Sdf.ValueTypeNames.String).Set(model["classification"]["kind"])
    root_prim.CreateAttribute("class:fluid", Sdf.ValueTypeNames.String).Set(model["classification"]["fluid"])

    props = model["properties"]
    root_prim.CreateAttribute("prop:material", Sdf.ValueTypeNames.String).Set(str(props.get("material", "")))
    root_prim.CreateAttribute("prop:max_pressure_Pa", Sdf.ValueTypeNames.Double).Set(float(props["max_pressure_Pa"]))
    root_prim.CreateAttribute("prop:volume_m3", Sdf.ValueTypeNames.Double).Set(float(props["volume_m3"]))
    root_prim.CreateAttribute("prop:mass_kg", Sdf.ValueTypeNames.Double).Set(float(props["mass_kg"]))

    dims = props["dims_m"]
    root_prim.CreateAttribute("prop:dims_m", Sdf.ValueTypeNames.Double3).Set(
        Gf.Vec3d(float(dims[0]), float(dims[1]), float(dims[2]))
    )

    # Traceability (string is always safe)
    trace = model.get("trace", {})
    root_prim.SetCustomDataByKey("trace:sysml_element_id", str(trace.get("sysml_element_id", "")))

    # ✅ Requirements as a Vt.StringArray (fixes your error)
    set_customdata_requirements(root_prim, trace.get("requirements", []))

    # Ports: for sanity check, store as a single string to avoid nested-type issues
    root_prim.SetCustomDataByKey("ports", str(model.get("ports", [])))

    # Geometry reference
    geom_asset = model["assets"]["geometry"]
    geom_mesh = UsdGeom.Mesh.Define(stage, f"/{name}/geom")
    geom_mesh.GetPrim().GetReferences().AddReference(geom_asset, "/OxygenTank_Geom")

    # Material reference + binding
    mtl_asset = model["assets"]["material"]
    material_name = props.get("material", "Material")

    materials_scope = UsdGeom.Scope.Define(stage, f"/{name}/materials")
    materials_scope.GetPrim().GetReferences().AddReference(mtl_asset)

    bound_mat_path = Sdf.Path(f"/{name}/materials/Materials/{material_name}")
    bound_mat = UsdShade.Material.Get(stage, bound_mat_path)
    if bound_mat and bound_mat.GetPrim().IsValid():
        UsdShade.MaterialBindingAPI.Apply(geom_mesh.GetPrim()).Bind(bound_mat)
    else:
        geom_mesh.GetPrim().SetCustomDataByKey("materialHint", f"{mtl_asset}:/Materials/{material_name}")

    stage.SetDefaultPrim(root_prim)
    stage.GetRootLayer().Save()
    return component_path


def build_o2tank_usd_from_canonical_json(model: Dict[str, Any], *, component_path: str = "components/OxygenTank.usda"):
    geom_path = model["assets"]["geometry"]
    mtl_path = model["assets"]["material"]

    write_geometry_layer(model, geom_path)
    write_material_layer(model, mtl_path)
    write_component_layer(model, component_path)

    return {"geometry": geom_path, "material": mtl_path, "component": component_path}



# Example
if __name__ == "__main__":
    canonical = {
        "type": "Part",
        "id": "urn:lunarspaceport1:part:OxygenTank:001",
        "name": "OxygenTank",
        "classification": {"domain": "ECLSS", "kind": "tank", "fluid": "O2"},
        "properties": {
            "material": "Aluminum_2219",
            "max_pressure_Pa": 30000000,
            "volume_m3": 1.0,
            "mass_kg": 120.0,
            "dims_m": [2.0, 1.0, 1.0],
        },
        "assets": {
            "geometry": "assets/geom/o2_tank_geom.usda",
            "material": "assets/mtl/aluminum.usda",
        },
        "ports": [
            {"name": "inlet", "type": "fluid", "medium": "O2"},
            {"name": "outlet", "type": "fluid", "medium": "O2"},
        ],
        "trace": {"sysml_element_id": "...", "requirements": ["REQ-ECLSS-O2-001"]},
    }

    outputs = build_o2tank_usd_from_canonical_json(canonical, component_path="components/OxygenTank.usda")
    print("Wrote:", outputs)


Wrote: {'geometry': 'assets/geom/o2_tank_geom.usda', 'material': 'assets/mtl/aluminum.usda', 'component': 'components/OxygenTank.usda'}
